# Import Packages

In [1]:
import pandas as pd
import numpy as np
import turicreate as tc

# Read the Files and Preparing the Data

In [2]:
post = pd.read_csv('posts (1).csv')
user_ratings = pd.read_csv('userRatings.csv', usecols=['user_id', 'post_id', 'Ratings'])

Display the first 5 rows of the post dataset

In [3]:
post.head(5)

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,2019-08-25 19:39:01,2019-08-28 11:30:00,1,NaN,NaN,NaN
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,2019-08-25 19:44:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,2019-08-25 19:50:33,2019-08-28 11:30:00,1,NaN,NaN,NaN
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,2019-08-25 19:51:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,2019-08-25 19:51:39,2019-08-28 11:30:00,1,NaN,NaN,NaN


Drop this columns because they are not relevant to our model

In [4]:
col = ['tags', 'slug', 'created_at', 'updated_at', 'image', 'status_id', 'action', 'post_id']

postdf = post.drop(post[col], axis=1)

Read the first 5 cols of the new dataset

In [5]:
postdf.head()

,user_id,title,content
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ..."
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str..."


Display the first 5 rows of the user_ratings dataset

In [6]:
user_ratings.head()

,user_id,post_id,Ratings
0,1,995.0,5
1,1,355.0,2
2,1,996.0,4
3,1,996.0,5
4,1,996.0,4


Merge our postdf and user ratings df

In [7]:
merged_col = pd.merge(postdf, user_ratings, on='user_id')

Display the head of our merged dataframe

In [8]:
merged_col.head()

,user_id,title,content,post_id,Ratings
0,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",143.0,3
1,2191,What you wish to still learn,1. I want more insight on clodinary and so on,143.0,3
2,2191,What you wish should be improved,1. For now am OK with the setting,143.0,3
3,4,Good Afternoon People,Hey,999.0,4
4,4,Good Afternoon People,Hey,278.0,3


In [9]:
#number of users

users = post['user_id'].unique() ; print("number of users" , len(users))

number of users 308


In [10]:
#number of content titles

titles = post['title'].unique() ; print("Number of titles" , len(titles))

Number of titles 658


In [11]:
#train_data, test_data = train_test_split(merged_col, test_size = 0.20, random_state=0)

Convert merged data to csv

In [12]:
merged_col.to_csv('merged_data.csv')

Reading back the csv file and converting it to Sframe for use by the turicreate model

In [13]:
data = tc.SFrame.read_csv('merged_data.csv')

Finished parsing file /home/godwin/Downloads/luci/merged_data.csv

Parsing completed. Parsed 100 lines in 0.066728 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/godwin/Downloads/luci/merged_data.csv

Parsing completed. Parsed 622 lines in 0.035917 secs.

# Model 

Creating a train and validation data for our model

In [14]:
trainind_data, validation_data = tc.recommender.util.random_split_by_user(data, 'user_id', 'title')

Building the model using the ranking factorization recommender

In [15]:
rating_model = tc.ranking_factorization_recommender.create(trainind_data, 
                                                    user_id ='user_id',
                                                   item_id ='title',
                                                   target = 'Ratings')

Preparing data set.

Data has 505 observations with 8 users and 45 items.

Data prepared in: 0.056074s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.56173

Final training RMSE: 0.73646

Creating an item_similarity model, So we can compare the two models together

In [16]:
item_sim_model = tc.item_similarity_recommender.create(trainind_data, user_id='user_id', 
                                                       item_id='content', 
                                                       target='Ratings', similarity_type='cosine')

Warning: Ignoring columns X1, title, post_id;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 505 observations with 8 users and 51 items.

Data prepared in: 0.0071s

...

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 28.874ms                            | 1.5              | 0               |

| 29.166ms                            | 100              | 51              |

+-------------------------------------+------------------+-----------------+

# Compare Both Models

In [17]:
tc.recommender.util.compare_models(validation_data, [rating_model,item_sim_model], metric='rmse')

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.093347s

PROGRESS: Evaluate model M0

Overall RMSE: 0.7293434771821536

Per User RMSE (best)
+---------+-------------------+-------+
| user_id |        rmse       | count |
+---------+-------------------+-------+
|    3    | 0.597710671837544 |   43  |
+---------+-------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    1    | 1.0986132743423718 |   2   |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+---------------+-------------------+-------+
|     title     |        rmse       | count |
+---------------+-------------------+-------+
| Lucid is live | 0.128095969219987 |   1   |
+---------------+-------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+--------+--------------------+-------+
| title  |        rmse        | count |
+--------+--------------------+-------+
| second | 1.51679277599489

[{'rmse_by_user': Columns:
  	user_id	int
  	rmse	float
  	count	int
  
  Rows: 4
  
  Data:
  +---------+--------------------+-------+
  | user_id |        rmse        | count |
  +---------+--------------------+-------+
  |    2    | 0.7581293190616205 |   65  |
  |    3    | 0.597710671837544  |   43  |
  |    1    | 1.0986132743423718 |   2   |
  |    7    | 1.0072452340455411 |   7   |
  +---------+--------------------+-------+
  [4 rows x 3 columns], 'rmse_by_item': Columns:
  	title	str
  	rmse	float
  	count	int
  
  Rows: 29
  
  Data:
  +-------------------------------+---------------------+-------+
  |             title             |         rmse        | count |
  +-------------------------------+---------------------+-------+
  |         Lucid is back         |  0.1903897955833166 |   3   |
  |         Stressful Day         |  0.5052079459086466 |   2   |
  |      Hello New Followers      |  0.1423483287206761 |   2   |
  |      How did your day go?     | 0.217223805114419

We compared both models and we can see that the rating model performed better with a rmse of 0.71

# Recommender System Testing

*Making predictions based on different factors

1. Making recommendation for all users
    * This makes recommendations for all users seen during model creation
    * It returns the first 10 prediction

In [18]:
all_users_rank = rating_model.recommend()
all_users_rank

user_id,title,score,rank
2191,First,3.677562069710308,1
2191,Took a long while,3.6764345975629626,2
2191,third,3.6651996873356043,3
2191,second,3.6500476036257683,4
2191,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.6358499448783337,5
2191,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.5989237759299217,6
2191,html test,3.588717271205002,7
2191,this is a test post,3.587428017150575,8
2191,with picture,3.586749374497646,9
2191,Welcome to Lucid,3.5783167859576284,10


* k=3 arguement means that it should return 3 predictions for each users
* It can basically be any amount of recommendations we want the recommender system to produce

In [19]:
all_users_rank = rating_model.recommend(k=3)
all_users_rank

user_id,title,score,rank
2191,First,3.677562069710308,1
2191,Took a long while,3.6764345975629626,2
2191,third,3.6651996873356043,3
4,Took a long while,3.655878009837085,1
4,First,3.651079372134024,2
4,second,3.642135336626285,3
1,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.9072983689643084,1
1,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.804407706115359,2
1,html test,3.784994889613205,3
7,third,3.62337239049744,1


2.Making recommendations for new users without any information
    * created a new user with user_id = 20000
    * asked the rec_sys to recommend 3 articles for the user

In [20]:
rank_factor = rating_model.recommend(users=[20000], k=3)
rank_factor

user_id,title,score,rank
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.761588371921295,1
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.6954678591377674,2
20000,THE SUNFLOWER,3.658081411946529,3


* it can also return a list of users

In [21]:
rank_factor = rating_model.recommend(users=[20000, 21000, 22000], k=3)
rank_factor

user_id,title,score,rank
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.761588371921295,1
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.6954678591377674,2
20000,THE SUNFLOWER,3.658081411946529,3
21000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.761588371921295,1
21000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.6954678591377674,2
21000,THE SUNFLOWER,3.658081411946529,3
22000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.761588371921295,1
22000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.6954678591377674,2
22000,THE SUNFLOWER,3.658081411946529,3


In [22]:
rating_model.save('rating_model')

# Follower Recommender System

Read the data

In [23]:
following = pd.read_csv('nooffollowers1.csv')
following.head()

,Unnamed: 0,follower_id,name,username,email,no of follower
0,0,6,Seyi Onifade,xyluz,hngtechapps@gmail.com,273
1,273,3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,242
2,515,2,Elijah Okokon,DMatrix,okoelijah@gmail.com,29
3,544,7,Mark Essien,markessien,markessien@gmail.com,328
4,872,9,Edmund Ekott,26th_edmund,edmund.timfon@gmail.com,47


Convert the data into Sframe

In [24]:
following = tc.SFrame(following)
following.head()

Unnamed: 0,follower_id,name,username,email,no of follower
0,6,Seyi Onifade,xyluz,hngtechapps@gmail.com,273
273,3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,242
515,2,Elijah Okokon,DMatrix,okoelijah@gmail.com,29
544,7,Mark Essien,markessien,markessien@gmail.com,328
872,9,Edmund Ekott,26th_edmund,edmund.timfon@gmail.com,47
919,1,Eniayomi Oluwaseyi,eniayomi,oluwaseyieniayomi@gmail.com ...,15
934,11,Jenni Essien,j.naiaretti,j.naiaretti@gmail.com,22
956,20,albert oboh,albidosurest,albidosurest@gmail.com,12
968,86,Benjamin-Whyss Olumide,benjaminwhyss,benjaminwhyss@gmail.com,10
978,220,Toyosi Durotoye,corperevon,corperevon@gmail.com,24


# Build the followers Recommender System
* We used the factorization model so our recommender system does not become bias to recommending the most popular followers to all users.

In [27]:
model_item_sim = tc.factorization_recommender.create(following, 
                                          user_id='follower_id', 
                                          item_id='name', 
                                          target='no of follower')

Preparing data set.

Data has 878 observations with 878 users and 878 items.

Data prepared in: 0.011902s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 0.344799

Final training RMSE: 0.587169

# Testing the Followers Recommender System

1. Testing for all users using by recommending 3 users to our users.

In [28]:
rec_result_factorization = model_item_sim.recommend(k=3)
rec_result_factorization

follower_id,name,score,rank
6,Mark Essien,34.425139271316894,1
6,Nneamaka Chalokwu,32.96994753986395,2
6,Anita Chiz-Ugeh,27.574119888840084,3
3,Omolola Richard,29.05719860702551,1
3,Tiberius Mairura,26.95606263786352,2
3,Oluwasomidotun Odumosu,23.71923311859167,3
2,ukadike ochem,18.023843788204555,1
2,Jeffrey Ogah,16.35144200235403,2
2,Tiberius Mairura,16.22310068517721,3
7,Nneamaka Chalokwu,36.276560746727355,1


 2. Making recommendations for new users

In [29]:
rec_result_factorization_newUsers = model_item_sim.recommend(users=[20000], k=3)
rec_result_factorization_newUsers

follower_id,name,score,rank
20000,David Adedeji,14.691708408890133,1
20000,Mark Essien,14.686150394974117,2
20000,Seyi Onifade,14.438315712509517,3


In [30]:
model_item_sim.save('follower_rec')